In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

# Soru 1:
Find a data set and build a KNN Regression and an OLS regression. Compare the two. How similar are they? Do they miss in different ways?

In [5]:
cancer=load_breast_cancer()
cancer

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [17]:
df=pd.DataFrame(cancer.data, columns=cancer.feature_names)
df["target"]=cancer.target

df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0


In [23]:
Y=df.target
X=df.iloc[:,:30]

In [39]:
# Look at whether imbalanced data or not

print(sum(df["target"])/len(df["target"]*100))
print(1-(sum(df["target"])/len(df["target"]*100)))

0.6274165202108963
0.37258347978910367


In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42)

In [54]:
print(sum(Y_train)/len(Y_train*100))
print(sum(Y_test)/len(Y_test*100),"\n")

print(len(Y_train[Y_train==0]))
print(len(Y_train[Y_train==1]))
print(len(Y_train))

0.6285714285714286
0.6228070175438597 

169
286
455


In [59]:
log_reg=linear_model.LogisticRegression()
log_reg.fit(X_train,Y_train)

y_train_pred=log_reg.predict(X_train)
y_test_pred=log_reg.predict(X_test)

log_reg.score(X_test,Y_test)

0.9649122807017544

In [74]:
# Firstly, apply cross validation

CV=cross_validate(estimator=log_reg, 
                  X=X, 
                  y=Y, 
                  cv=5,
                 return_train_score=True)

print("Train set scores:",CV['train_score'])
print("Mean of train score:", CV["train_score"].mean())
print("Test set scores:", CV["test_score"])
print("Mean of test set scores:", CV["test_score"].mean())

Train set scores: [0.96043956 0.94505495 0.94505495 0.96263736 0.94736842]
Mean of train score: 0.9521110468478889
Test set scores: [0.92982456 0.93859649 0.96491228 0.95614035 0.95575221]
Mean of test set scores: 0.9490451793199813


In [91]:
X_train=sm.add_constant(X_train)

result=sm.OLS(Y_train,X_train).fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     49.81
Date:                Sat, 26 Sep 2020   Prob (F-statistic):          2.23e-119
Time:                        09:53:43   Log-Likelihood:                 28.738
No. Observations:                 455   AIC:                             4.525
Df Residuals:                     424   BIC:                             132.3
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       3.0563      0.476      6.420      0.000       2.121       3.992
mean radius                 0.1971      0.200      0.987      0.324      -0.195       0.590
mean texture               -0.0028      0.009     -0.311      0.756      -0.020       0.015
mean perimeter             -0.0228      0.029     -0.777      0.438      -0.080       0.035
mean area                  -0.0003      0.001     -0.545      0.586      -0.002       0.001
mean smoothness             0.4115      2.236      0.184      0.854      -3.984       4.806
mean compactness            5.0017      1.523      3.285      0.001       2.009       7.994
mean concavity             -1.0059      1.165     -0.864      0.388      -3.295       1.284
mean concave points        -4.9157      2.311     -2.127      0.034      -9.459      -0.372
mean symmetry               0.3384      0.826      0.409      0.682      -1.286       1.963
mean fractal dimension     -5.8143      6.306     -0.922      0.357     -18.209       6.580
radius error               -0.4323      0.344     -1.257      0.209      -1.108       0.244
texture error               0.0126      0.042      0.300      0.764      -0.070       0.095
perimeter error             0.0082      0.046      0.179      0.858      -0.082       0.099
area error                  0.0012      0.001      0.833      0.405      -0.002       0.004
smoothness error          -18.0785      7.036     -2.569      0.011     -31.909      -4.248
compactness error           2.2080      2.302      0.959      0.338      -2.317       6.733
concavity error             4.2738      1.406      3.039      0.003       1.510       7.038
concave points error      -18.1590      6.173     -2.942      0.003     -30.293      -6.025
symmetry error              1.1945      3.113      0.384      0.701      -4.924       7.313
fractal dimension error     3.0120     12.404      0.243      0.808     -21.369      27.393
worst radius               -0.2144      0.065     -3.303      0.001      -0.342      -0.087
worst texture              -0.0096      0.008     -1.218      0.224      -0.025       0.006
worst perimeter             0.0087      0.007      1.259      0.209      -0.005       0.022
worst area                  0.0010      0.000      2.629      0.009       0.000       0.002
worst smoothness           -0.1324      1.565     -0.085      0.933      -3.209       2.945
worst compactness          -0.7627      0.431     -1.770      0.077      -1.610       0.084
worst concavity            -0.6157      0.293     -2.104      0.036      -1.191      -0.040
worst concave points        1.3262      1.050      1.263      0.207      -0.738       3.390
worst symmetry             -1.0211      0.541     -1.888      0.060      -2.084       0.042
worst fractal dimension    -1.2736      2.702  

##### Modelin eğitim performansı (R-squared)= 0.779  Şimdi KNN uygulayalım

In [98]:
knn=KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train,Y_train)

predict_knn=knn.score(X_test,Y_test)
predict_knn

0.9298245614035088

In [100]:
cross=cross_validate(estimator=knn,
                    X=X_test,
                    y=Y_test,
                    cv=3,
                    return_train_score=True)

print(cv["test_score"].mean())

0.9490451793199813


##### Çapraz doğrulama sonrasında test datasının doğruluğu biraz düşmüştür. Logistic regresyon gibi sınıflandırma algoritmalarında OLS kullanılmaz.Çünkü zaten sonuç 1 ya da 0 olacağından hataların karelerinin toplamını en aza indirme gibi bir şey olmaz.

##### Least Square regression(OLS) is not built for binary classification, as logistic regression performs a better job at classifying data points and has a better logarithmic loss function as opposed to least squares regression. KNN algorithms ise used in either regression or classification problems; but OLS is used only in regression problems

# Soru 2:
What are potential problems with implementing kNN on a very large dataset?

* In large datasets, the cost of calculating the distance between the new point and each existing points is huge which degrades the performance of the algorithm.
* The KNN algorithm doesn't work well with high dimensional data because with large number of dimensions, it becomes difficult for the algorithm to calculate the distance in each dimension.
* We need to do feature scaling (standardization and normalization) before applying KNN algorithm to any dataset. If we don't do so, KNN may generate wrong predictions.
* KNN is sensitive to noise in the dataset. We need to manually impute missing values and remove outliers.

# Soru 3:
Describe the training set and test set accuracy as the value of K increases?<

In [112]:
from sklearn.metrics import accuracy_score

In [120]:
# If value of k increase, accuracy of the model increase. Let's test it:

knn=KNeighborsClassifier(n_neighbors=2, metric="minkowski")

knn.fit(X_train,Y_train)

result=knn.predict(X_test)

score_knn_test=knn.score(X_test,Y_test)
print("Test set accuracy: ",score_knn_test)

score_knn_train=knn.score(X_train,Y_train)
print("Train set accuracy:", score_knn_train)


# Accuracy değeri iki türlü bulunabilir: Ya direkt. score(X_train,Y_train) denilir; ya da accuracy_score(Y_test,y_test_pred)  denilebilir.
print("Test set accuracy by other way",accuracy_score(Y_test,result))


Test set accuracy:  0.9298245614035088
Train set accuracy: 0.9648351648351648
Test set accuracy by other way 0.9298245614035088


In [126]:
# If k=5;

knn_=KNeighborsClassifier(n_neighbors=5, metric="minkowski")

knn_.fit(X_train,Y_train)

print("Accuracy of test set for k=5 :", knn_.score(X_test,Y_test))
print("Accuracy of train set for k=5 :", knn_.score(X_train,Y_train))

Accuracy of test set for k=5 : 0.956140350877193
Accuracy of train set for k=5 : 0.9406593406593406


##### k=5 için train set'in doğruluğu düşmüş olup, test set'in doğruluğu artmıştır; dolayısıyla k=5 için overfit olmuştur diyebiliriz.

In [146]:
# To find the best k value, apply the GridSearchCV

from sklearn.model_selection import GridSearchCV

parametreler={"n_neighbors":np.arange(1,8,1)}

grid=GridSearchCV(estimator=knn,
                 param_grid=parametreler,
                 cv=10)

grid.fit(X,Y)

print("The best parameter:", grid.best_params_)
print("The best score:", grid.best_score_)

The best parameter: {'n_neighbors': 5}
The best score: 0.9297619047619046


# Soru 4:
Which one of the statements are true:
* KNN performs much better if all of the data have the same scale
* KNN works well with a small number of input variables (p), but struggles when the number of inputs is very large--True
* KNN makes no assumptions about the functional form of the problem being solved

# Soru 5:
Calculate the distance between A(1,3) and B(2,3) based on Euclidean and Manhattan

In [140]:
knn_euc=KNeighborsClassifier(n_neighbors=5, weights="distance" )

In [ ]:
# Euclidean distance is= sqrt(square(2-1)+square(3-3))

In [139]:
euclid=np.sqrt(np.square(2-1)+np.square(3-3))
euclid

1.0